In [2]:
#ライブラリの読み込み
import numpy as np
import numpy.random as random
import scipy as sp
import pandas as pd
from pandas import Series, DataFrame

%precision 3

#可視化用
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

階層型インデックス(便利！)

In [3]:
#3行3列のデータを作成し、インデックスとカラムを設定
hier_df = DataFrame(np.arange(9).reshape((3,3)),
                   index=[['a','a','b'],[1,2,2]],
                   columns=[['Osaka','Tokyo','Osaka'],['Blue','Red','Red']])
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [4]:
# indexに名前を付ける
hier_df.index.names = ['key1','key2']
# カラムに名前を付ける
hier_df.columns.names = ['city','color']

hier_df

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5
b    2        6     7     8

In [5]:
#カラムの絞り込み
hier_df['Osaka']

color      Blue  Red
key1 key2           
a    1        0    2
     2        3    5
b    2        6    8

In [6]:
#インデックスを軸にした集計

#階層ごとの要約統計量:行合計
hier_df.sum(level = 'key2', axis=0)

city,Osaka,Tokyo,Osaka
color,Blue,Red,Red
key2,,,
1,0,1,2
2,9,11,13


In [7]:
#列合計
hier_df.sum(level = 'color', axis=1)

color      Blue  Red
key1 key2           
a    1        0    3
     2        3    9
b    2        6   15

In [8]:
#インデックス要素の削除
hier_df.drop(['b'])

city      Osaka Tokyo Osaka
color      Blue   Red   Red
key1 key2                  
a    1        0     1     2
     2        3     4     5

データの結合

In [15]:
#データの準備
#data1
data1 = {'id':['100','101','102','103','104','106','108','110','111','113'],
        'city':['Tokyo','Osaka','Kyoto','Hokkaido','Tokyo','Tokyo','Osaka','Kyoto','Hokkaido','Tokyo'],
        'birth_year':[1990,1989,1992,1997,1982,1991,1988,1990,1995,1981],
        'name':['Hiroshi','Akiko','Yuki','Satoru','Steeve','Mituru','Aoi','Tarou','Suguru','Mitsuo']}

df1 = DataFrame(data1)

#data2
data2 = {'id':['100','101','102','105','107'],
        'math':[50,43,33,76,98],
        'english':[90,30,20,50,30],
        'sex':['M','F','F','M','M'],
        'index_num':[0,1,2,3,4]}

df2 = DataFrame(data2)

In [13]:
df1

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


In [14]:
df2

,id,math,english,sex,index_num
0,100,50,90,M,0
1,101,43,30,F,1
2,102,33,20,F,2
3,105,76,50,M,3
4,107,98,30,M,4


In [11]:
#内部結合(共通要素だけ取り出して結合)
print('・結合テーブル')
pd.merge(df1,df2,on='id')
#onでkeyを指定

・結合テーブル


,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50,90,M,0
1,101,Osaka,1989,Akiko,43,30,F,1
2,102,Kyoto,1992,Yuki,33,20,F,2


In [12]:
#全結合(全体の要素同士を結合する。結合する値がないとき欠損値NaNとなる。)
pd.merge(df1,df2,how = 'outer')
#全結合はパラメータをouterに設定

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990.0,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989.0,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992.0,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997.0,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982.0,Steeve,NaN,NaN,NaN,NaN
5,106,Tokyo,1991.0,Mituru,NaN,NaN,NaN,NaN
6,108,Osaka,1988.0,Aoi,NaN,NaN,NaN,NaN
7,110,Kyoto,1990.0,Tarou,NaN,NaN,NaN,NaN
8,111,Hokkaido,1995.0,Suguru,NaN,NaN,NaN,NaN
9,113,Tokyo,1981.0,Mitsuo,NaN,NaN,NaN,NaN


In [17]:
# indexによるマージ
pd.merge(df1,df2, left_index=True, right_on='index_num')
#left_indexとright_onをindex_numに指定することで、df1のindexとdf2のindex_numをkeyとして結合可能。

,id_x,city,birth_year,name,id_y,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,100,50,90,M,0
1,101,Osaka,1989,Akiko,101,43,30,F,1
2,102,Kyoto,1992,Yuki,102,33,20,F,2
3,103,Hokkaido,1997,Satoru,105,76,50,M,3
4,104,Tokyo,1982,Steeve,107,98,30,M,4


In [18]:
#左外部結合(df1の要素にあわせてdf2の要素を結合する)
pd.merge(df1,df2,how='left')
#how=leftを指定する

,id,city,birth_year,name,math,english,sex,index_num
0,100,Tokyo,1990,Hiroshi,50.0,90.0,M,0.0
1,101,Osaka,1989,Akiko,43.0,30.0,F,1.0
2,102,Kyoto,1992,Yuki,33.0,20.0,F,2.0
3,103,Hokkaido,1997,Satoru,NaN,NaN,NaN,NaN
4,104,Tokyo,1982,Steeve,NaN,NaN,NaN,NaN
5,106,Tokyo,1991,Mituru,NaN,NaN,NaN,NaN
6,108,Osaka,1988,Aoi,NaN,NaN,NaN,NaN
7,110,Kyoto,1990,Tarou,NaN,NaN,NaN,NaN
8,111,Hokkaido,1995,Suguru,NaN,NaN,NaN,NaN
9,113,Tokyo,1981,Mitsuo,NaN,NaN,NaN,NaN


In [19]:
#data3
data3 = {'id':['117','118','119','120','125'],
        'city':['Chiba','Kanagawa','Tokyo','Fukkuoka','Okinawa'],
        'birth_year':[1990,1989,1992,1997,1982],
        'name':['Suguru','Kouichi','Satoshi','Yukie','Akari']}

df3 = DataFrame(data3)
df3

,id,city,birth_year,name
0,117,Chiba,1990,Suguru
1,118,Kanagawa,1989,Kouichi
2,119,Tokyo,1992,Satoshi
3,120,Fukkuoka,1997,Yukie
4,125,Okinawa,1982,Akari


In [20]:
#縦結合(keyなしで、縦にデータをマージする。concatメソッドを用いる。)
concat_data = pd.concat([df1,df3])
concat_data

,id,city,birth_year,name
0,100,Tokyo,1990,Hiroshi
1,101,Osaka,1989,Akiko
2,102,Kyoto,1992,Yuki
3,103,Hokkaido,1997,Satoru
4,104,Tokyo,1982,Steeve
5,106,Tokyo,1991,Mituru
6,108,Osaka,1988,Aoi
7,110,Kyoto,1990,Tarou
8,111,Hokkaido,1995,Suguru
9,113,Tokyo,1981,Mitsuo


データの操作と変換

In [23]:
#3行3列のデータを作成し、インデックスとカラムを設定
hier_df = DataFrame(np.arange(9).reshape((3,3)),
                   index=[['a','a','b'],[1,2,2]],
                   columns=[['Osaka','Tokyo','Osaka'],['Blue','Red','Red']])
hier_df

Osaka Tokyo Osaka
     Blue   Red   Red
a 1     0     1     2
  2     3     4     5
b 2     6     7     8

In [25]:
#ピボット操作:行と列の入れ替え
#「Blue,Red」の列を行に変換
hier_df.stack()

Osaka  Tokyo
a 1 Blue      0    NaN
    Red       2    1.0
  2 Blue      3    NaN
    Red       5    4.0
b 2 Blue      6    NaN
    Red       8    7.0

In [26]:
#ustackメソッドで、「Blue,Red」の行を列に変換
hier_df.stack().unstack()

Osaka     Tokyo     
     Blue Red  Blue  Red
a 1     0   2   NaN  1.0
  2     3   5   NaN  4.0
b 2     6   8   NaN  7.0

In [27]:
#重複データの除去

#重複のあるデータの作成
dupli_data = DataFrame({'col1':[1,1,2,3,4,4,6,6],
                       'col2':['a','b','b','b','c','c','b','b']})
print('・元のデータ')
dupli_data

・元のデータ


,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
5,4,c
6,6,b
7,6,b


In [28]:
#重複判定
dupli_data.duplicated()

0    False
1    False
2    False
3    False
4    False
5     True
6    False
7     True
dtype: bool

In [29]:
#重複削除
dupli_data.drop_duplicates()

,col1,col2
0,1,a
1,1,b
2,2,b
3,3,b
4,4,c
6,6,b


In [30]:
#マッピング処理(共通のキーとなるデータに対して、一方のテーブルからキーに対応するデータを引っ張ってくる機能)
#参照データ
city_map = {'Tokyo':'Kanto',
            'Hokkaido':'Hokkaido',
            'Osaka':'Kansai',
            'Kyoto':'Kansai'}
city_map

{'Tokyo': 'Kanto',
 'Hokkaido': 'Hokkaido',
 'Osaka': 'Kansai',
 'Kyoto': 'Kansai'}

In [31]:
#参照データを結合
#もし対応するデータがなかったらNaNになる
df1['region'] = df1['city'].map(city_map)
df1

,id,city,birth_year,name,region
0,100,Tokyo,1990,Hiroshi,Kanto
1,101,Osaka,1989,Akiko,Kansai
2,102,Kyoto,1992,Yuki,Kansai
3,103,Hokkaido,1997,Satoru,Hokkaido
4,104,Tokyo,1982,Steeve,Kanto
5,106,Tokyo,1991,Mituru,Kanto
6,108,Osaka,1988,Aoi,Kansai
7,110,Kyoto,1990,Tarou,Kansai
8,111,Hokkaido,1995,Suguru,Hokkaido
9,113,Tokyo,1981,Mitsuo,Kanto


In [33]:
#無名関数とmapを組み合わせる
#要素を一つ一つ取り出すより便利

#birth_yearの上3つの数字・文字を取り出す
df1['up_two_num'] = df1['birth_year'].map(lambda x: str(x)[0:3])
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,Kansai,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,Kansai,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


In [36]:
#ビン分割(pandasのcut関数でビンごとに分割可能)

#分割の粒度
birth_year_bins = [1980,1985,1990,1995,2000]

#ビン分割の実施
birth_year_cut_data = pd.cut(df1.birth_year,birth_year_bins)
birth_year_cut_data

0    (1985, 1990]
1    (1985, 1990]
2    (1990, 1995]
3    (1995, 2000]
4    (1980, 1985]
5    (1990, 1995]
6    (1985, 1990]
7    (1985, 1990]
8    (1990, 1995]
9    (1980, 1985]
Name: birth_year, dtype: category
Categories (4, interval[int64]): [(1980, 1985] < (1985, 1990] < (1990, 1995] < (1995, 2000]]

In [37]:
#集計結果
pd.value_counts(birth_year_cut_data)

(1985, 1990]    4
(1990, 1995]    3
(1980, 1985]    2
(1995, 2000]    1
Name: birth_year, dtype: int64

In [40]:
#名前を付ける(labelsで指定)
group_names = ['early1980s','late1980s','early1990s','late1990s']

birth_year_cut_data = pd.cut(df1.birth_year,birth_year_bins, labels = group_names)
pd.value_counts(birth_year_cut_data)

late1980s     4
early1990s    3
early1980s    2
late1990s     1
Name: birth_year, dtype: int64

In [41]:
#数字で分割数指定可能。ここでは２つに分割
pd.cut(df1.birth_year,2)

0      (1989.0, 1997.0]
1    (1980.984, 1989.0]
2      (1989.0, 1997.0]
3      (1989.0, 1997.0]
4    (1980.984, 1989.0]
5      (1989.0, 1997.0]
6    (1980.984, 1989.0]
7      (1989.0, 1997.0]
8      (1989.0, 1997.0]
9    (1980.984, 1989.0]
Name: birth_year, dtype: category
Categories (2, interval[float64]): [(1980.984, 1989.0] < (1989.0, 1997.0]]

In [42]:
#分位点での分割も可能
pd.value_counts(pd.qcut(df1.birth_year,2))

(1980.999, 1990.0]    6
(1990.0, 1997.0]      4
Name: birth_year, dtype: int64

データの集約とグループ演算

In [43]:
#データの用意(ただし、region付)
df1

,id,city,birth_year,name,region,up_two_num
0,100,Tokyo,1990,Hiroshi,Kanto,199
1,101,Osaka,1989,Akiko,Kansai,198
2,102,Kyoto,1992,Yuki,Kansai,199
3,103,Hokkaido,1997,Satoru,Hokkaido,199
4,104,Tokyo,1982,Steeve,Kanto,198
5,106,Tokyo,1991,Mituru,Kanto,199
6,108,Osaka,1988,Aoi,Kansai,198
7,110,Kyoto,1990,Tarou,Kansai,199
8,111,Hokkaido,1995,Suguru,Hokkaido,199
9,113,Tokyo,1981,Mitsuo,Kanto,198


In [44]:
#サイズ情報
df1.groupby('city').size()

city
Hokkaido    2
Kyoto       2
Osaka       2
Tokyo       4
dtype: int64

In [45]:
#cityを軸にbirth_yearの平均値を求める
df1.groupby('city')['birth_year'].mean()

city
Hokkaido    1996.0
Kyoto       1991.0
Osaka       1988.5
Tokyo       1986.0
Name: birth_year, dtype: float64

In [46]:
#as_index=Falseでインデックスが設定されなくなる
df1.groupby(['region','city'],as_index=False)['birth_year'].mean()

,region,city,birth_year
0,Hokkaido,Hokkaido,1996.0
1,Kansai,Kyoto,1991.0
2,Kansai,Osaka,1988.5
3,Kanto,Tokyo,1986.0


In [48]:
#groupbyにはイテレータというメソッドがある。
#イテレータ：反復的に値を取り出す
for group,subdf in df1.groupby('region'):
    print('=====================================================')
    print('Region Name:{0}'.format(group))
    print(subdf)

Region Name:Hokkaido
    id      city  birth_year    name    region up_two_num
3  103  Hokkaido        1997  Satoru  Hokkaido        199
8  111  Hokkaido        1995  Suguru  Hokkaido        199
Region Name:Kansai
    id   city  birth_year   name  region up_two_num
1  101  Osaka        1989  Akiko  Kansai        198
2  102  Kyoto        1992   Yuki  Kansai        199
6  108  Osaka        1988    Aoi  Kansai        198
7  110  Kyoto        1990  Tarou  Kansai        199
Region Name:Kanto
    id   city  birth_year     name region up_two_num
0  100  Tokyo        1990  Hiroshi  Kanto        199
4  104  Tokyo        1982   Steeve  Kanto        198
5  106  Tokyo        1991   Mituru  Kanto        199
9  113  Tokyo        1981   Mitsuo  Kanto        198


In [49]:
#複数の計算を行う(aggメソッド)
#カウント、平均、最大、最小を計算

student_data_math = pd.read_csv('./sample_data/student-mat.csv',sep=';')

#列に複数の関数を適応
functions = ['count','mean','max','min']
grouped_student_math_data1 = student_data_math.groupby(['sex','address'])
grouped_student_math_data1['age','G1'].agg(functions)

<ipython-input-49-a2c6576c24c3>:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_student_math_data1['age','G1'].agg(functions)


age                       G1                   
            count       mean max min count       mean max min
sex address                                                  
F   R          44  16.977273  19  15    44  10.295455  19   6
    U         164  16.664634  20  15   164  10.707317  18   4
M   R          44  17.113636  21  15    44  10.659091  18   3
    U         143  16.517483  22  15   143  11.405594  19   5

In [53]:
student_data_math

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,9,9,9
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,14,16,16
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,10,8,7
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,11,12,10
